In [ ]:
!pip install pycm==0.8.1

In [ ]:
import librosa.display
import scipy.io.wavfile as wavfile
import numpy
import os.path
from os import walk
from scipy import stats
import numpy as np
import librosa 
import numpy as np
from scipy.stats import norm
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
# Import the libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from pycm import *

def extractMelSpectrogram_features(folder):
    hop_length = 512
    n_fft = 2048
    n_mels = 128
    types = ["abnormal", "normal"]
    a = []
    b = []
    for nametype in types:
        _wavs = []
        wavs_duration = []
        for (_,_,filenames) in walk(folder+nametype+"/"):
            _wavs.extend(filenames)
            break
        Mel_Spectrogram = []
        for _wav in _wavs:
            # read audio samples
            if(".wav" in _wav): 
                file = folder +nametype+"/"+_wav
                print ("-"+file)
                signal, rate = librosa.load(file)  
                #The Mel Spectrogram
                S = librosa.feature.melspectrogram(signal, sr=rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
                S_DB = librosa.power_to_db(S, ref=np.max)
                #Mel_Spectrogram.append(S_DB)
                #print (S_DB)
                S_DB = S_DB.flatten()[:1200]
                a.append(S_DB)
                if(nametype == "normal"):
                    b.append(0)
                elif (nametype == "abnormal"):
                    b.append(1)
    pickle.dump(a, open(folder+'mel_features_svm_a.data', 'wb'))
    pickle.dump(b, open(folder+'mel_features_svm_b.data', 'wb'))
    
    return a, b

def getPrediction(folder, clf):
    hop_length = 512
    n_fft = 2048
    n_mels = 128
    types = ["abnormal", "normal"]
    list_y_true = []
    list_y_pred = []
    for nametype in types:
        _wavs = []
        wavs_duration = []
        for (_,_,filenames) in walk(folder+nametype+"/"):
            _wavs.extend(filenames)
            break
        for _wav in _wavs:
            # read audio samples
            if(".wav" in _wav): 
                file = folder +nametype+"/"+_wav
                print ("-"+file)
                signal, rate = librosa.load(file)  
                #The Mel Spectrogram
                S = librosa.feature.melspectrogram(signal, sr=rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
                S_DB = librosa.power_to_db(S, ref=np.max)
                S_DB = S_DB.flatten()[:1200]
                y_pred = clf.predict([S_DB])[0]
                if(nametype=="abnormal"):
                    y_true = 1
                else:
                    y_true = 0
                list_y_true.append(y_true)
                list_y_pred.append(y_pred)
                print ("- class: "+str(nametype)+" = "+str(y_true)+", prediction: "+str(y_pred))
    pickle.dump(list_y_true, open(folder+'list_y_true.data', 'wb'))
    pickle.dump(list_y_pred, open(folder+'list_y_pred_svm_prediction.data', 'wb'))
    return list_y_true, list_y_pred
    #
#---
folder = "/Users/mac/Documents/proposal_training_heartbeat/course1/datasets/ourdataset/train/"
types = ["abnormal", "normal"]
#a, b = extractMelSpectrogram_features(folder)
print ("#Load mel features:")
a = pickle.load(open(folder+'mel_features_svm_a.data', 'rb'))
b = pickle.load(open(folder+'mel_features_svm_b.data', 'rb'))
print ("#SVM:")
clf = svm.SVC()
clf = svm.SVC(kernel="rbf")
#clf.fit(a, b)
list_y_true, list_y_pred = getPrediction(folder, clf)

list_y_true = pickle.load(open(folder+'list_y_true.data', 'rb'))
list_y_pred = pickle.load(open(folder+'list_y_pred_svm_prediction.data', 'rb'))

print(classification_report(list_y_true, list_y_pred, target_names=types))

results = folder+"report.txt"
f = open(results, "w")
cm = ConfusionMatrix(actual_vector=list_y_true, predict_vector=list_y_pred) # Create CM From Data
f.write("{}\n".format(cm))
f.close()



#Load mel features:
#SVM:
              precision    recall  f1-score   support

    abnormal       0.79      1.00      0.89      2926
      normal       1.00      0.11      0.21       858

    accuracy                           0.80      3784
   macro avg       0.90      0.56      0.55      3784
weighted avg       0.84      0.80      0.73      3784



In [ ]:
#Split 3 folds
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
# importing shutil module  
import shutil

def copyTrain(data, k):
    for i in range(len(data)):
        source = data[i]
        destination = data[i].replace("train/", "train/train_kfold"+str(k))
        #dest = shutil.move(source, destination)
        dest = shutil.copy(source, destination) 

def copyTest(data, k):
    for i in range(len(data)):
        source = data[i]
        destination = data[i].replace("train/", "train/test_kfold"+str(k))
        #dest = shutil.move(source, destination)
        dest = shutil.copy(source, destination) 
        
cross_number = 3
kf = KFold(cross_number, True, 10)
folder = "/Users/mac/Documents/proposal_training_heartbeat/course1/datasets/ourdataset/train/"
normal = np.array(glob(folder+'/normal/*.wav'))
abnormal = np.array(glob(folder+'/abnormal/*.wav'))

#normal
kf.get_n_splits(normal)
i=1
for normal_train_index, normal_test_index in kf.split(normal):
    normal_train = normal[normal_train_index]
    normal_test = normal[normal_test_index]


    if not os.path.exists(folder+"train_kfold"+str(i)) and not os.path.exists(folder+"test_kfold"+str(i)):
        os.mkdir(folder+"train_kfold"+str(i))
        os.mkdir(folder+"test_kfold"+str(i))

    if (not os.path.exists(folder+"train_kfold"+str(i)+"/normal") and not os.path.exists(folder+"test_kfold"+str(i)+"/normal")):
        os.mkdir(folder+"train_kfold"+str(i)+"/normal")
        os.mkdir(folder+"test_kfold"+str(i)+"/normal")

    copyTrain(normal_train, i)
    copyTest(normal_test, i)
    
    i+=1
    
#abnormal
kf.get_n_splits(abnormal)
i=1
for abnormal_train_index, abnormal_test_index in kf.split(abnormal):
    abnormal_train = abnormal[abnormal_train_index]
    abnormal_test = abnormal[abnormal_test_index]


    if not os.path.exists(folder+"train_kfold"+str(i)) and not os.path.exists(folder+"test_kfold"+str(i)):
        os.mkdir(folder+"train_kfold"+str(i))
        os.mkdir(folder+"test_kfold"+str(i))

    if (not os.path.exists(folder+"train_kfold"+str(i)+"/abnormal") and not os.path.exists(folder+"test_kfold"+str(i)+"/abnormal")):
        os.mkdir(folder+"train_kfold"+str(i)+"/abnormal")
        os.mkdir(folder+"test_kfold"+str(i)+"/abnormal")

    copyTrain(abnormal_train, i)
    copyTest(abnormal_test, i)
    
    i+=1
    
